# Activity 2: Analysis of Latin documents for word-co-occurrence

In [ ]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [2]:
import re
import time

In [3]:
from pyspark import SparkContext, SparkConf
from itertools import combinations

In [4]:
conf = SparkConf().setAppName("Spark Word_Co")
sc = SparkContext(conf=conf)

## Code to read lemma csv file

In [5]:
lemmas = {}

In [6]:
def f1(seq):
   myset = set(seq)
   return list(myset)

In [7]:
f = open('/home/hadoop/csv/la.lexicon.csv', 'r')
while True:
        line=f.readline()
        if not line: break
        global lemmas
        word, crap, lemma = line.split(",")
        if word.strip().lower() not in lemmas:
            lemmas[word.lower().strip()] = list()
            lemmas[word.lower().strip()].append(lemma.lower().strip())
        lemmas[word.lower().strip()].append(lemma.lower().strip())
        
for key, value in lemmas.iteritems():
    lemmas[key] = f1(value)

## Function to reproduce lemmas for Bigrams

In [ ]:
def form_bigram_lemma(data):
    words = data[0]
    locations = data[1]
    maha_result = []
    lem_words0 = lemmas.get(re.sub('[^A-Za-z0-9]+', '', words[0])) 
    lem_words1 = lemmas.get(re.sub('[^A-Za-z0-9]+', '', words[1]))
    maha_result.append(data)
    
    if lem_words0 is not None and lem_words1 is not None:
        for word0 in lem_words0:
            for word1 in lem_words1:
                result = []
                result.append(word0)
                result.append(word1)
                res = (result, locations)
                maha_result.append(res)
    return maha_result

## Function to reproduce lemmas for Trigrams

In [ ]:
def form_trigram_lemma(data):
    words = data[0]
    locations = data[1]
    maha_result = []
    lem_words0 = lemmas.get(re.sub('[^A-Za-z0-9]+', '', words[0]))   
    lem_words1 = lemmas.get(re.sub('[^A-Za-z0-9]+', '', words[1]))
    lem_words2 = lemmas.get(re.sub('[^A-Za-z0-9]+', '', words[2]))
    maha_result.append(data)
    if lem_words0 is not None and lem_words1 is not None and lem_words2 is not None:
        for word0 in lem_words0:
            for word1 in lem_words1:
                for word2 in lem_words2:
                    result = []
                    result.append(word0)
                    result.append(word1)
                    result.append(word2)
                    res = (result, locations)
                    maha_result.append(res)
    return maha_result

## Function to read a line and split into words

In [40]:
def parse(data):
    t = data.split(" ")
    if len(t) > 1:
        if len(data.split(">")) > 1 and len(data.split(">")) < 3:
            loc, line = data.split(">")
            line = (line).replace(",","").replace(".","").replace("-","").replace("\t","").lower().encode('utf-8')   
            line_list = line.split(" ")
            return loc.encode('utf-8'), line_list
        else:
            return "", ""
    else:
        return "", ""

In [46]:
text_file = sc.textFile("/home/hadoop/input")

## For Bigrams

In [ ]:
timer = time.time()

sentence = text_file.map(parse) \
            .flatMap(lambda x : [((x[1][i],x[1][j]),x[0]+">")  for i in range(0,len(x[1])) for j in range(i+1,len(x[1]))])
      
bigrams_freq = sentence.combineByKey(lambda v:[v],lambda x,y:x+[y],lambda x,y:x+y)

lemma_bigrams = bigrams_freq.map(form_bigram_lemma)

lemma_bigrams.saveAsTextFile("/home/hadoop/bioutput")
getTime = time.time()-timer
print('\n It took: ' + str(getTime)+ 'seconds to complete')

## For Trigrams

In [ ]:
timer = time.time()
sentence = text_file.map(parse) \
    .flatMap(lambda x : [((x[1][i],x[1][j], x[1][j+1]),x[0]+">")  for i in range(0,len(x[1])) for j in range(i+1,len(x[1])-1)])

trigrams_freq = sentence.combineByKey(lambda v:[v],lambda x,y:x+[y],lambda x,y:x+y) 

lemma_trigrams = trigrams_freq.map(form_trigram_lemma)

lemma_trigrams.saveAsTextFile("/home/hadoop/trioutput")

getTime = time.time()-timer
print('\n It took: ' + str(getTime)+ 'seconds to complete')